In [203]:
%matplotlib notebook
%reload_ext autoreload
%autoreload 2

"""
This notebook just does everything in ./sh/preprocess.sh script
in an interactive manner.
"""

import sys
import os

sys.path.append('../')
sys.path.append('../paretoviz')
from paretoviz.utils import fmt
from paretoviz.utils import vectorops as vops

from paretoviz import normalizer as nrm
from paretoviz import tradeoff_calculator as tro
from paretoviz import peeler as plr
from paretoviz import palettizer as pltz

# Do all these plots with centroid based coloring

# rawfpath = "../data/spherical/spherical-3d.out"
# rawfpath = "../data/spherical/spherical-4d.out"
# rawfpath = "../data/spherical/spherical-8d.out"

# rawfpath = "../data/knee/knee-3d.out"
# rawfpath = "../data/knee/knee-4d.out"
# rawfpath = "../data/knee/knee-8d.out"

# rawfpath = "../data/line-surface/line-3d.out"
# rawfpath = "../data/line-surface/line-4d.out"
# rawfpath = "../data/line-surface/line-6d.out" # fails
# rawfpath = "../data/line-surface/line-8d.out"

# rawfpath = "../data/carcrash/carcrash-3d.out"
# rawfpath = "../data/carcrash/carcrash-c1/carcrash-c1-3d.out"
# rawfpath = "../data/carcrash/carcrash-c2/carcrash-c2-3d.out"

# Do all these plots with constraint based coloring

# rawfpath = "../data/knee-const/knee-const-3d.out"
# rawfpath = "../data/knee-const/knee-const-4d.out"
# rawfpath = "../data/knee-const/knee-const-8d.out"

# rawfpath = "../data/knee-const-mod/knee-const-mod-3d.out"
# rawfpath = "../data/knee-const-mod/knee-const-mod-4d.out"

# rawfpath = "../data/isolated/isolated-3d.out"
# rawfpath = "../data/isolated/isolated-4d.out"
# rawfpath = "../data/isolated/isolated-8d.out"

# rawfpath = "../data/c2dtlz2/c2dtlz2-3d.out"
# rawfpath = "../data/c2dtlz2/c2dtlz2-4d.out"
# rawfpath = "../data/c2dtlz2/c2dtlz2-5d.out"
# rawfpath = "../data/c2dtlz2/c2dtlz2-8d.out"

# rawfpath = "../data/c2dtlz2/c2dtlz2-c1/c2dtlz2-c1-3d.out"
# rawfpath = "../data/c2dtlz2/c2dtlz2-c2/c2dtlz2-c2-3d.out"
# rawfpath = "../data/c2dtlz2/c2dtlz2-c3/c2dtlz2-c3-3d.out"
# rawfpath = "../data/c2dtlz2/c2dtlz2-c4/c2dtlz2-c4-3d.out"

# rawfpath = "../data/gaa-das/gaa-das-10d.out"
rawfpath = "../data/gaa-lhs/gaa-lhs-10d.out"

# some ML data sets
# rawfpath = "../data/airofoil/airofoil-5d.out"
# rawfpath = "../data/banknote/banknote-4d.out"
# rawfpath = "../data/blood/blood-4d.out" # fails
# rawfpath = "../data/cccp/cccp-4d.out"
# rawfpath = "../data/concrete/concrete-8d.out" # fails
# rawfpath = "../data/iris/iris-4d.out" 
# rawfpath = "../data/mammogram/mammogram-5d.out" # fails
# rawfpath = "../data/wil/wil-7d.out" # fails
# rawfpath = "../data/yeast/yeast-8d.out" #  fails 

path, rawfname = os.path.split(rawfpath)
normfname = rawfname.split('.')[0] + "-norm.out"
mufname = normfname.split('.')[0] + "-mu.out"

In [192]:
# This part does the normalizer.py operation

vals = fmt.load(rawfpath)

print("Normalizing {0:d} data points.".format(len(vals)))
vals_ = nrm.normalize(vals)
fmt.cat(vals_)

path, rawfile = os.path.split(rawfpath)
normfile = os.path.join(path, rawfile.split('.')[0] + "-norm.out")
print("Saving normalized data into {0:s} ...".format(normfile))
fmt.save(vals_, normfile)

Normalizing 2782 data points.
3.3646e-01	3.9415e-01	4.2634e-01	4.9363e-01	4.2179e-01	2.9069e-01	3.4382e-01	2.9474e-01
2.3419e-01	4.5708e-01	2.5286e-01	3.8056e-01	4.1755e-01	3.6496e-01	4.6075e-01	4.0597e-01
3.7925e-01	4.1410e-01	2.7990e-01	4.5259e-01	5.2811e-01	2.9546e-01	3.1211e-01	3.0627e-01
4.6787e-01	4.1314e-01	5.2792e-01	2.7349e-01	3.1062e-01	3.9971e-01	2.9671e-01	2.6665e-01
4.9756e-01	3.2004e-01	3.4600e-01	3.3615e-01	3.3286e-01	4.8362e-01	3.6802e-01	3.1343e-01
3.1797e-01	4.0591e-01	3.0944e-01	3.5265e-01	2.0207e-01	4.3177e-01	4.7557e-01	4.6997e-01
7.3663e-02	1.3200e-01	1.1584e-02	1.7055e-01	2.1454e-01	1.7678e-01	3.1507e-01	9.6427e-01
5.3085e-01	3.2262e-01	2.1427e-01	4.6908e-01	2.9931e-01	4.1378e-01	3.4345e-01	3.5984e-01
4.2129e-01	2.3127e-01	4.1378e-01	3.7096e-01	4.0644e-01	4.0616e-01	2.8495e-01	4.6007e-01
4.7786e-01	3.8942e-01	2.3408e-01	4.3065e-01	3.5754e-01	3.2886e-01	4.4723e-01	3.2611e-01
3.3223e-01	4.3256e-01	1.8630e-01	4.3822e-01	4.9027e-01	3.3963e-01	4.3383e-01	3.0075e-01
4.

In [193]:
# This part does the tradeoff_calculator.py operation

# epsilon = 0.33 # spherical
# epsilon = 0.25 # knee
# epsilon = 0.025 # line
# epsilon = 0.25 # carcrash
# epsilon = 0.125 # knee-const, mod
# epsilon = 0.33 # isolated
# epsilon = 0.33 # c2dtlz2-3d, 4d
epsilon = 0.15 # c2dtlz2-5d

normfpath = os.path.join(path, normfname)
points = fmt.load(normfpath)

mu = tro.compute_tradeoff(points, epsilon, normalize = False)
fmt.cat(mu)

path, normfname = os.path.split(normfpath)
mufpath = os.path.join(path, normfname.split('.')[0] + "-mu.out")
print("Saving tradeoff values to {0:s} ...".format(mufpath))
fmt.save(mu, mufpath)

Computing neighborhood ...
Computing tradeoff values with epsilon = 0.1500...
6.9266e-01
8.2358e-01
9.0039e-01
9.7567e-01
7.1716e-01
8.8862e-01
5.5055e-01
1.0219e+00
7.3353e-01
7.5755e-01
1.0129e+00
8.1450e-01
9.0197e-01
9.4780e-01
9.2145e-01
9.6396e-01
6.9777e-01
9.3555e-01
7.3710e-01
8.3065e-01
1.0264e+00
7.8216e-01
7.4152e-01
9.5418e-01
7.1807e-01
9.8186e-01
9.8497e-01
7.2437e-01
9.0603e-01
9.2818e-01
9.6152e-01
9.4727e-01
7.9678e-01
7.8297e-01
9.1967e-01
6.2434e-01
9.2923e-01
6.5292e-01
7.6930e-01
8.8842e-01
9.3073e-01
9.6897e-01
9.5577e-01
7.8812e-01
8.9912e-01
1.0185e+00
1.0244e+00
9.8234e-01
1.0111e+00
7.1803e-01
7.3507e-01
8.1283e-01
9.0915e-01
8.3608e-01
1.0133e+00
9.0414e-01
9.8095e-01
7.4133e-01
8.4699e-01
7.5739e-01
9.3613e-01
7.3120e-01
6.7895e-01
7.4812e-01
8.6994e-01
1.0388e+00
6.8119e-01
9.3320e-01
9.7857e-01
8.4346e-01
9.2699e-01
7.2620e-01
6.7813e-01
6.7970e-01
7.5378e-01
8.9641e-01
6.7895e-01
9.8729e-01
8.8245e-01
7.6565e-01
9.0342e-01
7.1378e-01
7.0522e-01
8.3950e-0

In [194]:
# This part does the peeler.py operation

mode = "default"
# mode = "no-project"

normfpath = os.path.join(path, normfname)
points = fmt.load(normfpath)

m = len(points[0])
print("Peeling data point cloud in {0:s} mode ...".format(mode))
if mode == "default":
    # the defualt mode will project the points 
    # on a simplex on the first quadrant
    ppoints = plr.project(points)
    cpoints = plr.collapse(ppoints, dim = m - 1)
    boundaries = plr.peel(cpoints)
elif mode == "no-project":
    boundaries = plr.peel(points)
fmt.cat(boundaries, dtype = 'int')

layerfpath = os.path.join(path, normfname.split('.')[0] + "-layers.out")
print("Saving {0:d} layers into {1:s} ...".format(len(boundaries), layerfpath))
fmt.save(boundaries, layerfpath, dtype = 'int')

Peeling data point cloud in default mode ...
Total points: 2782
Computing alpha-shape ...
alpha-shape done.
Layer 0: points added 89, points left 2693.
Computing alpha-shape ...
alpha-shape done.
Layer 1: points added 1694, points left 999.
Computing alpha-shape ...
alpha-shape done.
Layer 2: points added 742, points left 257.
Computing alpha-shape ...
alpha-shape done.
Layer 3: points added 229, points left 28.
Computing alpha-shape ...
alpha-shape done.
Layer 4: points added 28, points left 0.
512	6	2058	1560	2587	1564	1567	2598	2608	2611	2613	2629	2633	2128	2132	84	601	2139	2141	2663	2156	621	622	114	1650	116	1655	1658	1663	1153	1667	1670	2694	2187	2703	2202	667	2721	1710	2747	2751	1737	1234	2776	237	754	1267	1785	1789	2314	787	2329	1817	292	1829	807	814	314	1349	1869	333	1365	1368	1370	2402	355	2408	1385	364	1393	1401	1927	1435	1951	1953	420	940	1974	2488	445	958	1473	449	2010	2019	489	1008	2546	499
1	2	3	5	7	10	11	12	13	14	15	17	19	20	23	25	26	28	29	30	31	33	34	36	39	40	41	42	43	4

In [206]:
# This part does the palettizer.py operation

n_layers = 4
mode = "default"
mode = "star"
mode = "stardecay"

# tester()
normfpath = os.path.join(path, normfname)
points = fmt.load(normfpath)

layerfile = normfname.split('.')[0] + "-layers.out"
layers = fmt.load(os.path.join(path, layerfile), dtype = 'int')

print("Computing palette coordinates with {0:d} layers in {1:s} mode ...".format(n_layers, mode))
if mode == "default":
    palette_coords = pltz.palettize(points, layers, n_layers = n_layers)
    palettefpath = os.path.join(path, normfname.split('.')[0] + "-palette.out")
elif mode == "polar":
    palette_coords = pltz.palettize_polar(points, layers, n_layers = n_layers)
    palettefpath = os.path.join(path, normfname.split('.')[0] + "-palette-polar.out")
elif mode == "logistic":
    palette_coords = pltz.palettize_logistic(points, layers, n_layers = n_layers)
    palettefpath = os.path.join(path, normfname.split('.')[0] + "-palette-logistic.out")
elif mode == "star":
    palette_coords = pltz.palettize_star(points, layers, n_layers = n_layers)
    palettefpath = os.path.join(path, normfname.split('.')[0] + "-palette-star.out")
elif mode == "stardecay":
    palette_coords = pltz.palettize_stardecay(points, layers, n_layers = n_layers)
    palettefpath = os.path.join(path, normfname.split('.')[0] + "-palette-stardecay.out")
else:
    print("Error: unknown mode \'{0:s}\'\n".format(mode))
    sys.exit(1)

print("Saving palette coordinates into {0:s} ...".format(palettefpath))
pltz.save_palette(palette_coords, palettefpath)

Computing palette coordinates with 4 layers in stardecay mode ...
Spreading with e^(-4.00x)
Saving palette coordinates into ../data/gaa-lhs/gaa-lhs-10d-norm-palette-stardecay.out ...
